In [1]:
import pandas as pd

In [2]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('/kaggle/input/tokenizer')

In [4]:
import torch
model = torch.load('/kaggle/input/model-7/model_distilbert.pt')

In [5]:
data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [17]:
from nltk.corpus import stopwords
import re
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split() 
    words = [word.lower() for word in words if word.isalpha()] 
    words = [word for word in words if word not in stop_words]  
    return ' '.join(words)

In [18]:
data['clean_text'] = data['text'].apply(clean_text)

In [30]:
data

,id,prompt_id,text,clean_text
0,0000aaaa,2,Aaa bbb ccc.,aaa bbb ccc
1,1111bbbb,3,Bbb ccc ddd.,bbb ccc ddd
2,2222cccc,4,CCC ddd eee.,ccc ddd eee


In [19]:
texts = data['clean_text'] 

In [20]:
texts = texts.tolist()

In [21]:
predictions = []

In [22]:
import torch.nn.functional as F

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [24]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [25]:
model.eval()
with torch.no_grad():
    for text in texts:
        enc = tokenizer(text, truncation=True, padding=True, return_tensors='pt')
        input_ids = enc['input_ids'].to(device)
        attention_mask = enc['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        
        logits = outputs.logits
        logits = F.softmax(logits)
        predictions.append(logits[0][1].item())

/tmp/ipykernel_42/3647664330.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(logits)


In [26]:
predictions

[0.9999045133590698, 0.9999043941497803, 0.9999045133590698]

In [27]:
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

In [28]:
sub['id'] = data['id']
sub['generated'] = predictions

In [29]:
sub.to_csv('submission.csv',index=False)